<a href="https://colab.research.google.com/github/BillyGun27/Deep-Learning-Model-Test/blob/master/Transformer_Nemo_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update && apt-get install -y libsndfile1 ffmpeg
!pip install Cython
!pip install nemo_toolkit['all']

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [71.4 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:14 http://security.ubuntu.com/

In [2]:
import nemo
import nemo.collections.nlp as nemo_nlp
nmt_model = nemo_nlp.models.machine_translation.MTEncDecModel.from_pretrained(model_name="nmt_en_fr_transformer12x2")

[NeMo W 2023-02-07 04:45:35 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-02-07 04:45:38 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-02-07 04:45:39 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-02-07 04:45:40 experimental:27] Module <class 'nemo.collections.asr.models.audio_to_audio_model.AudioToAudioModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-02-07 04:45:41 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is ex

[NeMo I 2023-02-07 04:45:42 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/nmt_en_fr_transformer12x2/versions/1.0.0rc1/files/nmt_en_fr_transformer12x2.nemo to /root/.cache/torch/NeMo/NeMo_1.15.0/nmt_en_fr_transformer12x2/2bfb7a511c3c2b948055cb2664b97c06/nmt_en_fr_transformer12x2.nemo
[NeMo I 2023-02-07 04:47:25 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-02-07 04:47:35 tokenizer_utils:179] Getting YouTokenToMeTokenizer with model: /tmp/tmpu15ta_2s/tokenizer.75.32000.BPE.model with r2l: False.
[NeMo I 2023-02-07 04:47:35 tokenizer_utils:179] Getting YouTokenToMeTokenizer with model: /tmp/tmpu15ta_2s/tokenizer.75.32000.BPE.model with r2l: False.


[NeMo W 2023-02-07 04:47:35 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    src_file_name: /raid/batches.tokens.75.16000.pkl
    tgt_file_name: /raid/batches.tokens.75.16000.pkl
    tokens_in_batch: 16000
    clean: true
    max_seq_length: 512
    cache_ids: false
    cache_data_per_node: false
    use_cache: false
    shuffle: true
    num_samples: -1
    drop_last: false
    pin_memory: false
    num_workers: 8
    load_from_cached_dataset: true
    reverse_lang_direction: false
    load_from_tarred_dataset: false
    metadata_path: null
    tar_shuffle_n: 100
    
[NeMo W 2023-02-07 04:47:35 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation 

[NeMo I 2023-02-07 04:47:47 save_restore_connector:243] Model MTEncDecModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.15.0/nmt_en_fr_transformer12x2/2bfb7a511c3c2b948055cb2664b97c06/nmt_en_fr_transformer12x2.nemo.


In [15]:
from nemo.collections.nlp.modules.common.transformer import (
    BeamSearchSequenceGenerator,
    BeamSearchSequenceGeneratorWithLanguageModel,
    EnsembleBeamSearchSequenceGenerator,
)

nmt_model.beam_search = BeamSearchSequenceGenerator(
                embedding=nmt_model.decoder.embedding,
                decoder=nmt_model.decoder.decoder,
                log_softmax=nmt_model.log_softmax,
                bos=nmt_model.decoder_tokenizer.bos_id,
                pad=nmt_model.decoder_tokenizer.pad_id,
                eos=nmt_model.decoder_tokenizer.eos_id,
                max_sequence_length=nmt_model.decoder.max_sequence_length,
                beam_size=4,
                len_pen=0.6,
                max_delta_length=5,
            )

In [27]:
best_translations = nmt_model.translate(
            text=['watch and learn'],
            source_lang='en',
            target_lang='fr',
            return_beam_scores=True,
            log_timing=True,
        )

#all_results, scores, best_translations = (
#    best_translations[0],
#    best_translations[1],
#    best_translations[2],
#)

best_translations

(['regarder et apprendre',
  'regarder et apprendre',
  'regarder et apprendre',
  'Regarder et apprendre'],
 [-1.152836561203003,
  -3.186513662338257,
  -3.5806145668029785,
  -3.8147964477539062],
 ['regarder et apprendre'],
 {'encoder': 0.022141456604003906,
  'sampler': 0.046280860900878906,
  'mean_src_length': 6.0,
  'mean_tgt_length': 5.0})